In [1]:
# install BeautifulSoup and requests for web scaping
!pip install BeautifulSoup4
!pip install requests
print('BeautifualSoup and requests installed')

BeautifualSoup and requests installed


In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### obtain the data
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [3]:
# get data from wiki
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(wikiurl).text

# use beautifulsoup to parse HTML/XML codes
soup = BeautifulSoup(source,'lxml')

#print(soup)

In [4]:
# get the table
data = []
columns = []
table = soup.find(class_='wikitable')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #first row as column
    if (index == 0):
        columns = section
    else:
        data.append(section)
#data

In [5]:
df = pd.DataFrame(data = data, columns = columns)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df = df[df['Borough'] != 'Not assigned']
#df

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace=True)
#df

In [8]:
df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df.shape

(103, 3)

## Use the Geocoder package or the csv file to create the the dataframe with latitude and logitude

In [10]:
!wget -q -O "Geospatial_Coordinates.csv" https://cocl.us/Geospatial_data
print('csv downloaded')
geodf = pd.read_csv('Geospatial_Coordinates.csv')

csv downloaded


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
lldf_temp = df.set_index('Postal Code')
geodf_temp = geodf.set_index('Postal Code')
df_geo = pd.concat([lldf_temp, geodf_temp], axis=1, join='inner')

df_geo.index.name = 'PostalCode'
df_geo.reset_index(inplace=True)

df_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
